### 查看每日已经采集到的space_url,对原始的space_urls做清洗

In [22]:
from pandas import DataFrame, read_excel, read_csv
from os import chdir, getcwd
from numpy import setdiff1d

In [7]:
chdir('./data')
getcwd()

'F:\\Github\\PyLearn\\craw_data\\hihonor\\data'

#### 分析上个采集时间点采集的数据包含的uid个数

In [8]:
user_info1 = read_excel('user_info1.xlsx', dtype={'uid':str})
print(len(user_info1))
user_info1.head()

45078


,uid,user_name,user_lv,province,active_value,post_num,replies,friends,total_sign,continue_sign,month_sign,last_sign,agg_score,last_score,sign_lv,medal_num,date
0,8282385,荣耀粉丝8282385,用户组 : LV10,广西,36074,183,6542,22,913,1,0,2021-01-08 20:21,12,4,[LV.10]以坛为家III,1,2022-12-04
1,30147236,荣耀粉丝30147236,用户组 : LV10,浙江,44076,178,8776,335,1642,1,0,2021-11-09 10:10,616,4,[LV.m],37,2022-12-04
2,12701964,荣耀粉丝12701964,用户组 : LV10,福建,674897,758,6978,1000,1576,2,0,2020-12-31 17:30,8,4,[LV.m],25,2022-12-04
3,39550353,荣耀粉丝39550353,用户组 : LV10,广东,84737,315,13793,31,975,37,0,2021-02-12 14:12,204,4,[LV.10]以坛为家III,8,2022-12-04
4,7433099,荣耀粉丝7433099,用户组 : LV9,马来西亚,19361,900,1374,222,84,1,0,2021-03-24 18:36,16,4,[LV.6]常住居民II,7,2022-12-04


In [21]:
uid_last = user_info1['uid'].unique()
print('上个时间采集的url数为: ', len(uid_last))

上个时间采集的url数为:  45070


#### 相对有信息的用户空间url

In [31]:
active_space_urls = user_info1.loc[:,['uid']]
# https://club.hihonor.com/cn/space-uid-212638451.html
active_space_urls['left'] = 'https://club.hihonor.com/cn/space-uid-'
active_space_urls['right'] = '.html'
active_space_urls['space_url'] = active_space_urls['left'] + active_space_urls['uid'] + active_space_urls['right']
active_space_urls.head()

,uid,left,right,space_url
0,8282385,https://club.hihonor.com/cn/space-uid-,.html,https://club.hihonor.com/cn/space-uid-8282385....
1,30147236,https://club.hihonor.com/cn/space-uid-,.html,https://club.hihonor.com/cn/space-uid-30147236...
2,12701964,https://club.hihonor.com/cn/space-uid-,.html,https://club.hihonor.com/cn/space-uid-12701964...
3,39550353,https://club.hihonor.com/cn/space-uid-,.html,https://club.hihonor.com/cn/space-uid-39550353...
4,7433099,https://club.hihonor.com/cn/space-uid-,.html,https://club.hihonor.com/cn/space-uid-7433099....


In [32]:
# 导出为活跃的space_urls_active.csv
active_space_urls.loc[:,['space_url']].to_csv('space_urls_active.csv', index=False, encoding='utf-8')
active_space_urls.head(1)

,uid,left,right,space_url
0,8282385,https://club.hihonor.com/cn/space-uid-,.html,https://club.hihonor.com/cn/space-uid-8282385....


##### 分析全部的空间space_urls

In [14]:
space_urls = read_csv('space_urls.csv')
print(len(space_urls))
space_urls.head()

106200


,space_url
0,https://club.hihonor.com/cn/space-uid-21263845...
1,https://club.hihonor.com/cn/space-uid-21357981...
2,https://club.hihonor.com/cn/space-uid-21981006...
3,https://club.hihonor.com/cn/space-uid-21946001...
4,https://club.hihonor.com/cn/space-uid-15900584...


In [18]:
space_urls['html'] = space_urls['space_url'].str.rsplit('-uid-').str.get(1)
space_urls['uid'] = space_urls['html'].str.split('.').str.get(0)
space_urls.head()

,space_url,uid,html
0,https://club.hihonor.com/cn/space-uid-21263845...,212638451,212638451.html
1,https://club.hihonor.com/cn/space-uid-21357981...,213579811,213579811.html
2,https://club.hihonor.com/cn/space-uid-21981006...,219810068,219810068.html
3,https://club.hihonor.com/cn/space-uid-21946001...,219460012,219460012.html
4,https://club.hihonor.com/cn/space-uid-15900584...,159005845,159005845.html


In [20]:
uid_all = space_urls['uid'].unique()
print(len(uid_all))
print(type(uid_all))

106188
<class 'numpy.ndarray'>


##### 由于上个时间采集的url肯定市uid_all的子集, 所以我们做差集运算获取未采集的uid

In [25]:
test = setdiff1d([1,2,3,4],[3,4])
print(type(test))
print(test)

<class 'numpy.ndarray'>
[1 2]


In [27]:
uid_others = setdiff1d(uid_all, uid_last, assume_unique=True)
print('未采集uid数: ', len(uid_others))

未采集uid数:  61118


In [28]:
other_space_urls = space_urls.set_index('uid').loc[uid_others]
other_space_urls.head() # 经过测试, 这里的url基本为无效用户

,space_url,html
uid,,
219810068,https://club.hihonor.com/cn/space-uid-21981006...,219810068.html
219795973,https://club.hihonor.com/cn/space-uid-21979597...,219795973.html
219461796,https://club.hihonor.com/cn/space-uid-21946179...,219461796.html
217543772,https://club.hihonor.com/cn/space-uid-21754377...,217543772.html
219806064,https://club.hihonor.com/cn/space-uid-21980606...,219806064.html


In [33]:
other_space_urls.loc[:,['space_url']].to_csv('space_urls_sleep.csv', index=False, encoding='utf-8')